In [1]:
import csv
import keras
import pyspark
import numpy as np
from pyspark.sql import SparkSession
from keras.preprocessing.text import Tokenizer
#from pyspark.ml.feature import NGram
import re
import cPickle as pickle

Using TensorFlow backend.


In [2]:
sc = pyspark.SparkContext(appName="preprocessor")
ss = SparkSession(sc)

In [3]:
with open('tokenizer.pickle', 'rb') as handle:
    t = pickle.load(handle)
max_tweet_length = 280

In [4]:
def get_tweets(row):
    return row[3]

def remove_urls(tweet):
    return re.sub(r"http\S+", "", tweet)

In [5]:
rdd = ss.read.csv("Sentiment Analysis Dataset.csv").rdd.map(list)
header = rdd.first()
rdd = rdd.filter(lambda line: line != header)

In [6]:
# rdd.take(5)

tweets_rdd = rdd.map(lambda line: get_tweets(line).encode('utf-8'))
tweets_rdd = tweets_rdd.map(lambda line: remove_urls(line).lstrip(' '))
tweets_rdd.first()

'is so sad for my APL friend.............'

In [7]:
vocab_size = len(t.word_index) 
digits_rdd = tweets_rdd.map(lambda line: t.texts_to_sequences([line]))


In [8]:
digits_rdd.first()

[[8, 15, 108, 11, 5, 141442, 258]]

In [9]:
def pad(seq):
    return pad_sequences(seq, maxlen=max_tweet_length, padding='post')

padded_rdd = digits_rdd.map(lambda line: pad(line))

In [ ]:



ss.stop()
    

In [ ]:
# tweets_DF = rdd.filter(lambda row : row != header).toDF(header)





# tweets_list = rdd.collect()
# t.fit_on_texts(tweets_list)
# print(tweets_list[0])
#digits_rdd = tweets_rdd.map(lambda line: t.fit_on_texts(line))

# digits_rdd = tweets_rdd.map(lambda line: t.text_to_word_sequence(line))
# vocab_size = len(t.word_index) + 1
# digits_rdd = tweets_rdd.map(lambda line: t.text_to_word_sequence(line))
# digits_rdd.take(5)